
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [6]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [4]:
# number of callbacks for balck-sounding names
sum(data[data.race=='b'].call)

157.0

###### What test is appropriate for this problem? Does CLT apply?

In [7]:
data.head(5)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [8]:
race_call_data = data[['race','call']]
race_call_data_white = race_call_data[race_call_data.race=='w']
race_call_data_black = race_call_data[race_call_data.race!='w']

print(len(race_call_data_black))
print(len(race_call_data_white))

2435
2435


These are clearly binomial distributions. Let $\hat{p_w}$ be the sample probability of sucess (being called back) for a white person and let $\hat{p_b}$ be the corresponding measure for a black person. Then:

In [12]:
p_w = len(race_call_data_white[race_call_data_white.call==1])/len(race_call_data_white)
p_b = len(race_call_data_black[race_call_data_black.call==1])/len(race_call_data_black)

print(p_w)
print(p_b)

0.09650924024640657
0.06447638603696099


I have to check whether or not they meet the conditions to be approximated by a normal distribution. This is boils down to verifying if $np \ge 10$ and if $n(1-p) \ge 10$.

In [14]:
print(len(race_call_data_white)*p_w)
print(len(race_call_data_white)*(1-p_w))

print(len(race_call_data_black)*p_b)
print(len(race_call_data_black)*(1-p_b))

235.0
2200.0
157.0
2278.0


This proves that CLT applies.

###### What are the null and alternate hypotheses?

The null hypothesis is that race--or this data's equivalent representation--has no impact on the whether or not an applicant is called back. The alternative hypothesis is that it does. Mathematically:

$H_0: p_B = p_W $

$H_A: p_B \neq p_W$

I want a 95% confidence interval so the test statistics is 1.96. The margin of error in this case is given by:

$Z_{1-\frac{\alpha}{2}} \sqrt{\frac{\hat{p_1}(1-\hat{p_1})}{n_1}+\frac{\hat{p_2}(1-\hat{p_2})}{n_2}}$

In [17]:
import math
z = 1.96
error_margin = z*math.sqrt(p_w*(1-p_w)/len(race_call_data_white)+p_b*(1-p_b)/len(race_call_data_black))
error_margin

0.015255406349886438

The confidence interval is:

$\hat{p_1}-\hat{p_2}  \pm (error \quad margin)$

In [18]:
[p_w-p_b-z*error_margin,
 p_w-p_b+z*error_margin]

[0.0021322577636681654, 0.061933450655223]

Zero's absence from this interval is ominous. The p-value is:

In [19]:
from statsmodels.stats.proportion import proportions_ztest as pz
white_call = len(race_call_data_white[race_call_data_white.call==1])
black_call = len(race_call_data_black[race_call_data_black.call==1])

pz(np.array([white_call,black_call]),np.array([len(race_call_data_white),len(race_call_data_black)]),value=0)

(4.1084121524343464, 3.9838868375850767e-05)

The p-value is the second item in the tuple. It's very small which means that I would reject the null hypothesis.
By doing some research (read googling) I discovered another to compute this significance that I never heard of before. Its name is Fisher's exact test. I've decided to use that method instead. To use it I need a 2x2 contingency table which calls for some data transformation.

In [20]:
freq_table = pd.crosstab(race_call_data.race,race_call_data.call)
freq_table

call,0.0,1.0
race,,
b,2278,157
w,2200,235


In [21]:
from scipy.stats import fisher_exact
fisher_exact(freq_table)

(1.5498841922408801, 4.7587471079089322e-05)

Here again, null hypothesis is rejected

###### Discuss statistical significance.

Since I rejected the null hypothesis due to the small p-value, this means that race does have an impact on the rate at which applicants are accepted for interviews by employers.